In [ ]:
# following along with a long twitch stream exploring MuZero

In [91]:
%pylab inline
import tensorflow as tf
import numpy as np
import gym
import OpenGL
from tqdm import tqdm, trange
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
np.set_printoptions(suppress=True)

def to_one_hot(x,n):
  ret = np.zeros([n])
  ret[x] = 1.0
  return ret

def bstack(bb):
    ret = [[x] for x in bb[0]]
    for i in range(1, len(bb)):
        for j in range(len(bb[i])):
            ret[j].append(bb[i][j])
    return [np.array(x) for x in ret]

Populating the interactive namespace from numpy and matplotlib


In [92]:
# from muzero.model import MuModel

In [93]:

env = gym.make("CartPole-v0")
#env = gym.make("MountainCar-v0")
#env = gym.make("LunarLander-v2")
#env = gym.make("Acrobot-v1")
env.observation_space, env.action_space

(Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32),
 Discrete(2))

In [94]:
S_DIM = 4

# h: representation function
# 

x = o_0 = Input(env.observation_space.shape)
x = Dense(S_DIM)(x)
s_0 = Activation('tanh')(x)
unh = Model(o_0, s_0, name="h")
def ht(o_0):
    return h.predict(o_0[None])[0]

# g: dynamics function (recurrent in state?) old_state+action -> state + reward
# takes in the state s_k-1, and the action of k and outputs the reward
# r_k, s_k = g(s_k-1, a_k)

s_kml = Input(S_DIM)
a_k = Input(env.action_space.n)
x = Concatenate()([s_kml, a_k])
x = Dense(64)(x)
x = Activation('elu')(x)
x = Dense(64)(x)
x = Activation('elu')(x)
# state
s_k = Dense(S_DIM, name='s_k')(x)
# reward
r_k = Dense(1, name='r_k')(x)
g = Model([s_kml, a_k], [r_k, s_k], name="g")
g.compile('adam', 'mse')
def gt(s_kml, a_k):
    r_k, s_k = g.predict([s_kml[None], a_k[None]])
    return r_k[0], s_k[0]


x = s_k = Input(S_DIM)
x = Dense(32)(x)
x = Activation('elu')(x)
p_k = Dense(env.action_space.n)(x)
p_k = Activation('softmax', name='p_k')(p_k)
v_k = Dense(1, name='v_k')(x)
f = Model(s_k, [p_k, v_k], name="f")
f.compile('adam', 'mse')
def ft(s_kml, a_k):
    p_k, v_k = f.predict(s_k[None])
    return p_k[0], v_k[0]

In [95]:
# will use the dynamics function for rollout search
K = 5
gamma = 0.95

# represent
o_0 = Input(env.observation_space.shape, name="o_0")
# dont use the h function rn
# s_kml = h(o_0)
s_kml = o_0

# rollout with dynamics
# p_k, v_k, r_k = mu(o_0, a_l_k)
# inialization of a_all mu_all

a_all, mu_all = [], []

for k in range(K):
    a_k = Input(env.action_space.n, name="a_%d" % k)
    r_k, s_k = g([s_kml, a_k])
    
    # predict
    p_k, v_k = f([s_k])
    
    # store
    a_all.append(a_k)
    mu_all.append([p_k, v_k, r_k])
    s_kml = s_k
    
# put in the first observation and actions
#  need policy for search
#  need values from the sum of records + last state value (real state?)
# a_all = Concatenate()(a_all)
mu = Model([o_0, a_all], mu_all)
mu.compile('adam', 'mse')
mu.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
o_0 (InputLayer)                [(None, 4)]          0                                            
__________________________________________________________________________________________________
a_0 (InputLayer)                [(None, 2)]          0                                            
__________________________________________________________________________________________________
g (Functional)                  [(None, 1), (None, 4 4933        o_0[0][0]                        
                                                                 a_0[0][0]                        
                                                                 g[0][1]                          
                                                                 a_1[0][0]                  

In [99]:
import itertools

aopts = list(itertools.product([0,1], repeat=K))
aoptss = np.array([[to_one_hot(x, 2) for x in aa] for aa in aopts])
aoptss = aoptss.swapaxes(0,1)
aoptss = [aoptss[x] for x in range(5)]

def search(o_0):
    # 2 ^ 5 rollout isn't too bad, explore them all
    # maximize value
    o_0s = np.repeat(np.array(o_0)[None], len(aopts), axis=0)
    ret = mu.predict([o_0s]+aoptss)
    v = [(ret[-2][i][0], aopts[i]) for i in range(len(ret[-2]))]
    return sorted(v, reverse=True)

env.reset()


array([0.04796711, 0.04587159, 0.01429969, 0.0194777 ])

In [100]:
# test the dynamics model
env.reset()
for _ in range(10):
    env.render()
    a_k = env.action_space.sample()
    r_k, s_k = gt(env.state, to_one_hot(a_k, env.action_space.n))
    p_k, v_k = ft(s_k)
    print(a_k, r_k, v_k, np.argmax(p_k))
    env.unwrapped.state = s_k
#env.close

ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

In [101]:
env.render()

ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

1.5.0
<module 'OpenGL.version' from '/opt/anaconda3/lib/python3.8/site-packages/OpenGL/version.py'>


In [102]:
import gym
env= gym.make('CartPole-v1')
img = env.render()

ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    